<h1><center>Segmenting and Clustering Neighborhoods in Toronto</center></h1> 

## Imports

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

## 1- Scraping data  (PART 1 of the assignment)

In [2]:
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df= pd.read_html(wiki_url)[0]


In [3]:
df

Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25          M8C      Not assigned   
26          M9C         Etobicoke   
27          M1E       Scarborough   
28          M2E      Not assigned   
29          M3E      Not assigned   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
33          M7E      Not assigned   
34          M8E      Not assigned   
35          M9E      Not assigned   
36          M1G       Scarborough   
37          M2G      Not assigned   
38          M3G      Not assigned   
39          M4G         East York   
40          M5G  Downtown Toronto   
41          M6G  Downtown Toronto   
42          M7G      Not assigned   
43          M8G      Not assigned   
44          M9G      Not assigned   
45          M1H       Scarborough   
46          M2H        North York   
47          M3H        North York   
48          M4H         East York   
49          M5H  Downtown Toronto   
50          M6H      West Toronto   
51          M7H      Not assigned   
52          M8H      Not assigned   
53          M9H      Not assigned   
54          M1J       Scarborough   
55          M2J        North York   
56          M3J        North York   
57          M4J         East York   
58          M5J  Downtown Toronto   
59          M6J      West Toronto   
60          M7J      Not assigned   
61          M8J      Not assigned   
62          M9J      Not assigned   
63          M1K       Scarborough   
64          M2K        North York   
65          M3K        North York   
66          M4K      East Toronto   
67          M5K  Downtown Toronto   
68          M6K      West Toronto   
69          M7K      Not assigned   
70          M8K      Not assigned   
71          M9K      Not assigned   
72          M1L       Scarborough   
73          M2L        North York   
74          M3L        North York   
75          M4L      East Toronto   
76          M5L  Downtown Toronto   
77          M6L        North York   
78          M7L      Not assigned   
79          M8L      Not assigned   
80          M9L        North York   
81          M1M       Scarborough   
82          M2M        North York   
83          M3M        North York   
84          M4M      East Toronto   
85          M5M        North York   
86          M6M              York   
87          M7M      Not assigned   
88          M8M      Not assigned   
89          M9M        North York   
90          M1N       Scarborough   
91          M2N        North York   
92          M3N        North York   
93          M4N   Central Toronto   
94          M5N   Central Toronto   
95          M6N              York   
96          M7N      Not assigned   
97          M8N      Not assigned   
98          M9N              York   
99          M1P       Scarborough   
100         M2P        North York   
101         M3P      Not assigned   
102         M4P   Central Toronto   
103         M5P   Central Toronto   
104         M6P      West Toronto   
105         M7P      Not assigned   
106         M8P      Not assigned   
107     

### Cleaning the data frame

1-Ignore cells with a borough that is Not assigned.

2-More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

3-If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


Let's rename our columns:


In [4]:
 df.rename(columns={'Postal Code':'PostalCode', 'Neighbourhood':'Neighborhood' }, inplace=True)

1- We'll drop the rows for which Borough is Not assigned:

In [5]:
index_names = df[df['Borough'] == "Not assigned" ].index 
df.drop(index_names, inplace = True) 

In [6]:
df.reset_index(drop=True,inplace=True)
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
30         M5H  Downtown Toronto   
31         M6H      West Toronto   
32         M1J       Scarborough   
33         M2J        North York   
34         M3J        North York   
35         M4J         East York   
36         M5J  Downtown Toronto   
37         M6J      West Toronto   
38         M1K       Scarborough   
39         M2K        North York   
40         M3K        North York   
41         M4K      East Toronto   
42         M5K  Downtown Toronto   
43         M6K      West Toronto   
44         M1L       Scarborough   
45         M2L        North York   
46         M3L        North York   
47         M4L      East Toronto   
48         M5L  Downtown Toronto   
49         M6L        North York   
50         M9L        North York   
51         M1M       Scarborough   
52         M2M        North York   
53         M3M        North York   
54         M4M      East Toronto   
55         M5M        North York   
56         M6M              York   
57         M9M        North York   
58         M1N       Scarborough   
59         M2N        North York   
60         M3N        North York   
61         M4N   Central Toronto   
62         M5N   Central Toronto   
63         M6N              York   
64         M9N              York   
65         M1P       Scarborough   
66         M2P        North York   
67         M4P   Central Toronto   
68         M5P   Central Toronto   
69         M6P      West Toronto   
70         M9P         Etobicoke   
71         M1R       Scarborough   
72         M2R        North York   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence 

2- It appears that Wikipedia already lists each postal code once and neighborhoods are separated by comma. We can double check this by observing how many times a postal code appears in the data frame. 

In [7]:
#for verification:value counts for each postal code is 1.
set(df['PostalCode'].value_counts())

{1}

In [8]:
# for verification: number of rows in the data frame is equal to the number of unique postal code values
len(df)==len(df['PostalCode'].unique())

True

3- Let's see if there are any neighborhood values that is "Not assigned"

In [9]:
index_names = df[df['Neighborhood'] == "Not assigned" ].index 
len(index_names)


0

We do not have any rows with a "Not assigned" neighborhood. 

The dataframe has 103 rows and 3 columns

In [10]:
print(df.shape)

(103, 3)


#### Since we'll be using Neighborhood column, let's check if there are any repetitions:

In [11]:
df['Neighborhood'].value_counts()

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
The Danforth West, Riverdale                                                                                                              1
Humberlea, Emery                                                                                                                          1
Forest Hill North & West, Forest Hill Road Park                                                                                           1
Golden Mile, Clairlea, Oakridge                                                                                                           1
Studio District                                                                                                                           1
Hillcrest Village   

It appears the neighborhood Don Mills appears twice, and Downsview appears 4 times. Let's check their corresponding postal codes:

In [12]:
downsview_index = df[df['Neighborhood'] == "Downsview" ].index 
donmills_index = df[df['Neighborhood'] == "Don Mills" ].index 
print("Downsview rows:", downsview_index )
print("Don Mills rows:", donmills_index )

Downsview rows: Int64Index([40, 46, 53, 60], dtype='int64')
Don Mills rows: Int64Index([7, 13], dtype='int64')


In [13]:
df.iloc[downsview_index]

PostalCode     Borough Neighborhood
40        M3K  North York    Downsview
46        M3L  North York    Downsview
53        M3M  North York    Downsview
60        M3N  North York    Downsview

In [14]:
df.iloc[donmills_index]

PostalCode     Borough Neighborhood
7         M3B  North York    Don Mills
13        M3C  North York    Don Mills

#### We'll rename these repeated Neighborhood names by including the postal code at the beginning.

In [16]:
#downsview
for i in downsview_index:
    df.iloc[i,2]=df.iloc[i,0]+" "+ df.iloc[i,2]
for i in donmills_index:
    df.iloc[i,2]=df.iloc[i,0]+" "+ df.iloc[i,2]

In [17]:
df.iloc[downsview_index]

PostalCode     Borough   Neighborhood
40        M3K  North York  M3K Downsview
46        M3L  North York  M3L Downsview
53        M3M  North York  M3M Downsview
60        M3N  North York  M3N Downsview

In [18]:
df.iloc[donmills_index]

PostalCode     Borough   Neighborhood
7         M3B  North York  M3B Don Mills
13        M3C  North York  M3C Don Mills

Let's verify that we have a unique neighborhood for each row:

In [21]:
df.shape[0]==df['Neighborhood'].nunique()

True

In [76]:
print(df.shape)

(103, 3)


## 2- Adding Lattitude and Longtitude data to our data frame (PART 2 of the assignment)

Even for one postal code, the geocoder could not return the langtitude and lontitude data in a reasonable time.
Thus, we'll get the data from this file: http://cocl.us/Geospatial_data

In [23]:
!wget -O Geospatial_data  http://cocl.us/Geospatial_data

--2020-11-13 12:13:02--  http://cocl.us/Geospatial_data
Resolving cocl.us... 169.63.96.176, 169.63.96.194
Connecting to cocl.us|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-11-13 12:13:02--  https://cocl.us/Geospatial_data
Connecting to cocl.us|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-13 12:13:03--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 107.152.25.197
Connecting to ibm.box.com|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-11-13 12:13:03--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection t

In [24]:
geo_data=pd.read_csv('Geospatial_data')
geo_data.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Let's remove the space in Postal Code:

In [25]:
geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

Let's create our final data frame, toronto_data, by merging our data frames based on the PostalCode column:

In [26]:
toronto_data=pd.merge(df, geo_data, on='PostalCode')
toronto_data.head()

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [80]:
toronto_data.head(15)

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A  Downtown Toronto   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   
12        M1C       Scarborough   
13        M3C        North York   
14        M4C         East York   

                                         Neighborhood   Latitude  Longitude  
0                                           Parkwoods  43.753259 -79.329656  
1                                    Victoria Village  43.725882 -79.315572  
2                           Regent Park, Harbourfront  43.654260 -79.360636  
3                    Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
5             Islington Avenue, Humber Valley Village  43.667856 -79.532242  
6                                      Malvern, Rouge  43.806686 -79.194353  
7                                       M3B Don Mills  43.745906 -79.352188  
8                     Parkview Hill, Woodbine Gardens  43.706397 -79.309937  
9                            Garden District, Ryerson  43.657162 -79.378937  
10                                          Glencairn  43.709577 -79.445073  
11  West Deane Park, Princess Gardens, Martin Grov...  43.650943 -79.554724  
12             Rouge Hill, Port Union, Highland Creek  43.784535 -79.160497  
13                                      M3C Don Mills  43.725900 -79.340923  
14                                   Woodbine Heights  43.695344 -79.318389

## 3- Exploring, Analyzing, Clustering, and Examining Neighborhoods in Toronto (PART 3 of the assignment)

There are 4 sub-sections for this part:

3A- Exploring neighborhoods in Toronto

3B- Analyzing neighborhoods in Toronto

3C- Clustering neighborhoods in Toronto

3D- Examining neighborhoods in Toronto


#### Assumptions:
In our dataframe, neighborhoods with the same postal code are listed together and appear as a group in the Neighborhood column. We'll assume each group is a single neighborhood. Also, some neighborhood names were paired with more than one postal code. We renamed those by including the postal code in the name and treating them as different neighborhoods in our analysis.

In [28]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The Toronto dataframe has 10 boroughs and 103 neighborhoods


In [29]:
toronto_data.shape

(103, 5)

### 3A- Exploring neighborhoods in Toronto

#### Use geopy library to get the latitude and longitude values of Toronto.

In [30]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [75]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'],toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare credentials

In [123]:

CLIENT_ID = 'my_client_id'
CLIENT_SECRET = 'my_client_secret'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: my_client_id
CLIENT_SECRET:my_client_secret


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i=0
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(i,name)
        i+=1
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        # foursquare server is sometimes down and that gives an error. whenever we get an error, we try calling the server again. 
 
        while True:
            try: 
                results = requests.get(url).json()["response"]['groups'][0]['items']
                break
            except:
                pass
            
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [35]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop

#### Let's check the size of the dataframe:

In [36]:
print(toronto_venues.shape)

(2139, 7)


In [37]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               5   
Alderwood, Long Branch                                                  7   
Bathurst Manor, Wilson Heights, Downsview North                        21   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      22   
Berczy Park                                                            55   
Birch Cliff, Cliffside West                                             4   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     68   
Christie                                                               16   
Church and Wellesley                                                   75   
Clarks Corners, Tam O'Shanter, Sullivan                                12   
Cliffside, Cliffcrest, Scarborough Village West                         2   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             33   
Davisville North                                                        9   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                        5   
Dorset Park, Wexford Heights, Scarborough Town ...                      5   
Dufferin, Dovercourt Village                                           13   
East Toronto, Broadview North (Old East York)                           3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                      8   
Fairview, Henry Farm, Oriole                                           70   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Glencairn                                                               5   
Golden Mile, Clairlea, Oakridge                                        10   
Guildwood, Morningside, West Hill                                       8   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
Hillcrest Village                                                       5   
Humber Summit                                                           2   
Humberlea, Emery                                                        1   
Humewood-Cedarvale                                                      5   
India Bazaar, The Beaches West                                         19   
Kennedy Park, Ionview, East Birchmount Park                             4   
Kensington Market, Chinatown, Grange Park                              74   
Kingsview Village, St. Phillips, Martin Grove G...                      4   
Lawrence Manor, Lawrence Heights                                       13   
Lawrence Park                                                           3   
Leaside                                                                33   
Little Portugal, Trinity                                               45   
M3B Don Mills                                                           4   
M3C Don Mills                                                          19   
M3K Downsview                                                           2   
M3L Downsview                                   

In [38]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 unique categories.


#### Let's save our data in a file to prevent calling the foursquare API as sometimes their server is down.

In [39]:
toronto_venues.to_csv('toronto_venues.csv',index=False)

### 3B- Analyzing Neighborhoods in Toronto

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


In [41]:
toronto_onehot.head()

Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0               0                 0                   0      0   
1               0                 0                   0      0   
2               0                 0                   0      0   
3               0                 0                   0      0   
4               0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1               0        0            0                0         0   
2               0        0            0                0         0   
3               0        0            0                0         0   
4               0        0            0                0         0   

   Burger Joint  Burrito Place  Bus Line  Bus Station  Business Service  \
0             0    

In [42]:
toronto_onehot.shape

(2139, 274)

#### It appears that there is a problem here. We had 274 unique venue categories. After adding neighborhood as a column, we should have 275 columns in our data frame.
This means that there is a venue category called 'Neighborhood'



In [43]:
index_names = toronto_venues[toronto_venues['Venue Category'] == "Neighborhood" ].index 
index_names

Int64Index([357, 604, 792, 1270], dtype='int64')

There are 4 indices returned. Let's print these rows:

In [44]:
toronto_venues[['Neighborhood','Venue','Venue Category' ]].iloc[index_names]

Neighborhood             Venue  \
357                                         The Beaches     Upper Beaches   
604                            Richmond, Adelaide, King  Downtown Toronto   
792   Harbourfront East, Union Station, Toronto Islands      Harbourfront   
1270                                    Studio District       Leslieville   

     Venue Category  
357    Neighborhood  
604    Neighborhood  
792    Neighborhood  
1270   Neighborhood

#### We need to create our toronto_onehot data frame again. This time, before adding our Neighborhood column, let's rename the venue category "Neighborhood" to "VenueCat_Neighborhood"

In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# change the venue category type Neighborhood to  "VenueCat_Neighborhood"
toronto_onehot.rename(columns={"Neighborhood": "VenueCat_Neighborhood"},inplace=True)

#Now we can add our original neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


In [46]:
toronto_onehot.shape

(2139, 275)

In [47]:
toronto_onehot.head()

Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0               0                 0                   0      0   
1               0                 0                   0      0   
2               0                 0                   0      0   
3               0                 0                   0      0   
4               0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1               0        0            0                0         0   
2               0        0            0                0         0   
3               0        0            0                0         0   
4               0        0            0                0         0   

   Burger Joint  Burrito Place  Bus Line  Bus Station  Business Service  \
0             0    

In [48]:
# move neighborhood column to the first column
col_name="Neighborhood"
first_col = toronto_onehot.pop(col_name)
toronto_onehot.insert(0, col_name, first_col)


In [49]:
toronto_onehot.head()


Neighborhood  Accessories Store  Afghan Restaurant  Airport  \
0         Parkwoods                  0                  0        0   
1         Parkwoods                  0                  0        0   
2  Victoria Village                  0                  0        0   
3  Victoria Village                  0                  0        0   
4  Victoria Village                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0     0    0               0                 0                   0      0   
1     0    0               0                 0                   0      0   
2     0    0               0                 0                   0      0   
3     0    0               0                 0                   0      0   
4     0    0               0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1               0        0            0                0         0   
2               0        0            0                0         0   
3               0        0            0                0         0   
4               0        0            0                0     

In [50]:
toronto_onehot['Neighborhood'].nunique()

100

We originally head 103 neighborhoods. Now, it appears we have 100 unique neighborhood values in our data frame. I think this is because there were no venues returned by the foursquare API for three neighborhoods.

We can alter the code for getNearbyVenues function such that it displays neighborhoods for which no venue is returned.

In [52]:
toronto_onehot['Neighborhood'].value_counts()

Toronto Dominion Centre, Design Exchange                                                                                                  100
Commerce Court, Victoria Hotel                                                                                                            100
Harbourfront East, Union Station, Toronto Islands                                                                                         100
First Canadian Place, Underground city                                                                                                    100
Richmond, Adelaide, King                                                                                                                  100
Garden District, Ryerson                                                                                                                  100
Stn A PO Boxes                                                                                                                             96
St. Ja

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Accessories Store  \
0                                           Agincourt           0.000000   
1                              Alderwood, Long Branch           0.000000   
2     Bathurst Manor, Wilson Heights, Downsview North           0.000000   
3                                     Bayview Village           0.000000   
4                   Bedford Park, Lawrence Manor East           0.000000   
5                                         Berczy Park           0.000000   
6                         Birch Cliff, Cliffside West           0.000000   
7        Brockton, Parkdale Village, Exhibition Place           0.000000   
8   Business reply mail Processing Centre, South C...           0.000000   
9   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
10                                Caledonia-Fairbanks           0.000000   
11              Canada Post Gateway Processing Centre           0.000000   
12                                          Cedarbrae           0.000000   
13                                 Central Bay Street           0.000000   
14                                           Christie           0.000000   
15                               Church and Wellesley           0.000000   
16            Clarks Corners, Tam O'Shanter, Sullivan           0.000000   
17    Cliffside, Cliffcrest, Scarborough Village West           0.000000   
18                     Commerce Court, Victoria Hotel           0.000000   
19                                         Davisville           0.000000   
20                                   Davisville North           0.000000   
21   Del Ray, Mount Dennis, Keelsdale and Silverthorn           0.000000   
22  Dorset Park, Wexford Heights, Scarborough Town...           0.000000   
23                       Dufferin, Dovercourt Village           0.000000   
24      East Toronto, Broadview North (Old East York)           0.000000   
25  Eringate, Bloordale Gardens, Old Burnhamthorpe...           0.000000   
26                       Fairview, Henry Farm, Oriole           0.000000   
27             First Canadian Place, Underground city           0.000000   
28    Forest Hill North & West, Forest Hill Road Park           0.000000   
29                           Garden District, Ryerson           0.000000   
30                                          Glencairn           0.000000   
31                    Golden Mile, Clairlea, Oakridge           0.000000   
32                  Guildwood, Morningside, West Hill           0.000000   
33  Harbourfront East, Union Station, Toronto Islands           0.000000   
34                      High Park, The Junction South           0.000000   
35                                  Hillcrest Village           0.000000   
36                                      Humber Summit           0.000000   
37                                   Humberlea, Emery           0.000000   
38                                 Humewood-Cedarvale           0.000000   
39                     India Bazaar, The Beaches West           0.000000   
40        Kennedy Park, Ionview, East Birchmount Park           0.000000   
41          Kensington Market, Chinatown, Grange Park           0.000000   
42  Kingsview Village, St. Phillips, Martin Grove ...           0.000000   
43                   Lawrence Manor, Lawrence Heights           0.076923   
44                                      Lawrence Park           0.000000   
45                                            Leaside           0.000000   
46                           Little Portugal, Trinity           0.000000   
47                                      M3B Don Mills           0.000000   
48                                      M3C Don Mills           0.000000   
49                                      M3K Downsview           0.000000   
50                                      M3L Downsview           0.000000   
51                                      M3M Downsview           0.000000   
52           

#### Let's confirm the new size. The number of rows should be equal to the number of neighborhoods (for which Foursquare provided venues) and number of columns should be equal to the number of unique venue categories plus 1.

In [54]:
toronto_grouped.shape

(100, 275)

#### Let's create a data frame that displays the top 10 venue categories for each neighborhood.
We first create a function to sort and pick the top k venues for a given row.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now, we display the top 10 venues for each neighborhood.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Agincourt        Clothing Store   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                  Bank   
3                                  Bayview Village   Japanese Restaurant   
4                Bedford Park, Lawrence Manor East        Sandwich Place   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Lounge        Breakfast Spot          Skating Rink   
1        Sandwich Place           Coffee Shop                   Pub   
2           Coffee Shop    Frozen Yogurt Shop    Chinese Restaurant   
3                  Café    Chinese Restaurant                  Bank   
4    Italian Restaurant           Coffee Shop         Grocery Store   

       5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0  Latin American Restaurant     Electronics Store      Doner Restaurant   
1                   Pharmacy                   Gym         Grocery Store   
2              Shopping Mall        Sandwich Place                 Diner   
3                Yoga Studio               Dog Run      Doner Restaurant   
4            Thai Restaurant             Locksmith          Liquor Store   

     8th Most Common Venue      9th Most Common Venue  \
0               Donut Shop                  Drugstore   
1            Deli / Bodega          Electronics Store   
2               Restaurant  Middle Eastern Restaurant   
3               Donut Shop                  Drugstore   
4  Comfort Food Restaurant                    Butcher   

        10th Most Common Venue  
0          Dumpling Restaurant  
1  Eastern European Restaurant  
2                  Supermarket  
3          Dumpling Restaurant  
4                    Juice Bar

### 3C- Clustering Neighborhoods in Toronto

Run _k_-means to cluster the neighborhood into 5 clusters.



In [57]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [58]:
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 4, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 2, 0, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
30         M5H  Downtown Toronto   
31         M6H      West Toronto   
32         M1J       Scarborough   
33         M2J        North York   
34         M3J        North York   
35         M4J         East York   
36         M5J  Downtown Toronto   
37         M6J      West Toronto   
38         M1K       Scarborough   
39         M2K        North York   
40         M3K        North York   
41         M4K      East Toronto   
42         M5K  Downtown Toronto   
43         M6K      West Toronto   
44         M1L       Scarborough   
45         M2L        North York   
46         M3L        North York   
47         M4L      East Toronto   
48         M5L  Downtown Toronto   
49         M6L        North York   
50         M9L        North York   
51         M1M       Scarborough   
52         M2M        North York   
53         M3M        North York   
54         M4M      East Toronto   
55         M5M        North York   
56         M6M              York   
57         M9M        North York   
58         M1N       Scarborough   
59         M2N        North York   
60         M3N        North York   
61         M4N   Central Toronto   
62         M5N   Central Toronto   
63         M6N              York   
64         M9N              York   
65         M1P       Scarborough   
66         M2P        North York   
67         M4P   Central Toronto   
68         M5P   Central Toronto   
69         M6P      West Toronto   
70         M9P         Etobicoke   
71         M1R       Scarborough   
72         M2R        North York   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  \
0                                            Parkwoods  43.753259 -79.329656   
1                                     Victoria Village  43.725882 -79.315572   
2                  

Let's print the rows with NaN values (these are the neighborhoods for which Foursquare did not return any venues). We should have 3 of these.

In [61]:
is_NaN = toronto_merged.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = toronto_merged[row_has_NaN]
print(rows_with_NaN)


   PostalCode      Borough                             Neighborhood  \
5         M9A    Etobicoke  Islington Avenue, Humber Valley Village   
52        M2M   North York                  Willowdale, Newtonbrook   
95        M1X  Scarborough                              Upper Rouge   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
5   43.667856 -79.532242             NaN                   NaN   
52  43.789053 -79.408493             NaN                   NaN   
95  43.836125 -79.205636             NaN                   NaN   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5                    NaN                   NaN                   NaN   
52                   NaN                   NaN                   NaN   
95                   NaN                   NaN                   NaN   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5                    NaN                   NaN                   NaN   
52               

Let's remove these rows from our data frame:

In [63]:
toronto_merged.drop([5,52,95],inplace=True)

In [65]:
toronto_merged

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
30         M5H  Downtown Toronto   
31         M6H      West Toronto   
32         M1J       Scarborough   
33         M2J        North York   
34         M3J        North York   
35         M4J         East York   
36         M5J  Downtown Toronto   
37         M6J      West Toronto   
38         M1K       Scarborough   
39         M2K        North York   
40         M3K        North York   
41         M4K      East Toronto   
42         M5K  Downtown Toronto   
43         M6K      West Toronto   
44         M1L       Scarborough   
45         M2L        North York   
46         M3L        North York   
47         M4L      East Toronto   
48         M5L  Downtown Toronto   
49         M6L        North York   
50         M9L        North York   
51         M1M       Scarborough   
53         M3M        North York   
54         M4M      East Toronto   
55         M5M        North York   
56         M6M              York   
57         M9M        North York   
58         M1N       Scarborough   
59         M2N        North York   
60         M3N        North York   
61         M4N   Central Toronto   
62         M5N   Central Toronto   
63         M6N              York   
64         M9N              York   
65         M1P       Scarborough   
66         M2P        North York   
67         M4P   Central Toronto   
68         M5P   Central Toronto   
69         M6P      West Toronto   
70         M9P         Etobicoke   
71         M1R       Scarborough   
72         M2R        North York   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  \
0                                            Parkwoods  43.753259 -79.329656   
1                                     Victoria Village  43.725882 -79.315572   
2                            Regent Park, Harbourfront  43.654260 -79.360636   
3                     Lawrence Manor, Lawrence 

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3D-Examining Clusters

**Cluster 1**  red colored on the map

It appears that "Park" is either the $1^{st}$ or $2^{nd}$ most common venue in this cluster.
This is what sets apart this cluster from the others. 
It is also easier to observe many other similarities in between the neighborhoods in this cluster.

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0         North York             0.0                  Park   
21              York             0.0                  Park   
35         East York             0.0          Intersection   
40        North York             0.0               Airport   
49        North York             0.0                Bakery   
61   Central Toronto             0.0                  Park   
64              York             0.0                  Park   
66        North York             0.0                  Park   
68   Central Toronto             0.0                 Trail   
77         Etobicoke             0.0           Pizza Place   
85       Scarborough             0.0            Playground   
91  Downtown Toronto             0.0                  Park   

   2nd Most Common Venue        3rd Most Common Venue  \
0      Food & Drink Shop                  Yoga Studio   
21         Women's Store                         Pool   
35                  Park            Convenience Store   
40                  Park                  Yoga Studio   
49                  Park   Construction & Landscaping   
61           Swim School                     Bus Line   
64           Yoga Studio  Eastern European Restaurant   
66     Convenience Store                  Yoga Studio   
68                  Park             Sushi Restaurant   
77                  Park               Sandwich Place   
85                  Park                       Bakery   
91            Playground                        Trail   

          4th Most Common Venue        5th Most Common Venue  \
0           Dumpling Restaurant          Distribution Center   
21          Dumpling Restaurant               Discount Store   
35  Eastern European Restaurant          Distribution Center   
40  Eastern European Restaurant          Distribution Center   
49                  Yoga Studio  Eastern European Restaurant   
61                  Yoga Studio                    Drugstore   
64          Distribution Center                      Dog Run   
66  Eastern European Restaurant          Distribution Center   
68                Jewelry Store                  Yoga Studio   
77                     Bus Line                   Donut Shop   
85                  Yoga Studio          Dumpling Restaurant   
91                  Yoga Studio                    Drugstore   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                Dog Run      Doner Restaurant            Donut Shop   
21   Distribution Center               Dog Run      Doner Restaurant   
35               Dog Run      Doner Restaurant            Donut Shop   
40               Dog Run      Doner Restaurant            Donut Shop   
49               Dog Run      Doner Restaurant            Donut Shop   
61   Distribution Center               Dog Run      Doner Restaurant   
64      Doner Restaurant            Donut Shop             Drugstore   
66               Dog Run      Doner Restaurant            Donut Shop   
68   Distribution Center               Dog Run      Doner Restaurant   
77        Discount Store   Distribution Center               Dog Run   
85   Distribution Center               Dog Run      Doner Restaurant   
91                 Diner        Discount Store   Distribution Center   

   9th Most Common Venue 10th Most Common Venue  
0              Drugstore      Electronics Store  
21            Donut Shop              Drugstore  
35             Drugstore    Dumpling Restaurant  
40             Drugstore    Dumpling Restaurant  
49             Drugstore    Dumpling Restaurant  
61            Donut Shop    Dumpling Restaurant  
64   Dumpling Restaurant      Electronics Store  
66             Drugstore    Dumpling Restaurant  
68            Donut Shop              Drugstore  
77      Doner Restaurant              Drugstore  
85            Donut Shop              Drugstore  
91               Dog Run       Doner Restaurant

**Cluster 2** purple colored on the map

Baseball Field is either the $1^{st}$ or $2^{nd}$ most common venue in this cluster.
A Baseball Field does not appear to be a venue in the top 10 for the other neighborhoods.

It is also interesting to see that if *Etobicoke* did not have *Construction and Landscaping* as the most common venue, these two neighborhoods would have exactly the same common venue categories in the top 9.

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
57   North York             1.0              Baseball Field   
101   Etobicoke             1.0  Construction & Landscaping   

    2nd Most Common Venue        3rd Most Common Venue  \
57            Yoga Studio  Eastern European Restaurant   
101        Baseball Field                  Yoga Studio   

           4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
57                       Dog Run      Doner Restaurant            Donut Shop   
101  Eastern European Restaurant               Dog Run      Doner Restaurant   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
57              Drugstore   Dumpling Restaurant     Electronics Store   
101            Donut Shop             Drugstore   Dumpling Restaurant   

    10th Most Common Venue  
57          Discount Store  
101      Electronics Store

**Cluster 3**

This is our biggest cluster in size. For the smaller ones, we can observe the similarities easier. It may help to increase the number of clusters or to define new features to get a better understanding.

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
1          North York             2.0                 Pizza Place   
2    Downtown Toronto             2.0                 Coffee Shop   
3          North York             2.0              Clothing Store   
4    Downtown Toronto             2.0                 Coffee Shop   
7          North York             2.0        Caribbean Restaurant   
8           East York             2.0                 Pizza Place   
9    Downtown Toronto             2.0                 Coffee Shop   
10         North York             2.0                 Pizza Place   
12        Scarborough             2.0  Construction & Landscaping   
13         North York             2.0                         Gym   
14          East York             2.0                Skating Rink   
15   Downtown Toronto             2.0                 Coffee Shop   
16               York             2.0                       Field   
17          Etobicoke             2.0                 Pizza Place   
18        Scarborough             2.0                Intersection   
19       East Toronto             2.0           Health Food Store   
20   Downtown Toronto             2.0                 Coffee Shop   
22        Scarborough             2.0                 Coffee Shop   
23          East York             2.0                 Coffee Shop   
24   Downtown Toronto             2.0                 Coffee Shop   
25   Downtown Toronto             2.0               Grocery Store   
26        Scarborough             2.0                 Gas Station   
27         North York             2.0                 Golf Course   
28         North York             2.0                        Bank   
29          East York             2.0           Indian Restaurant   
30   Downtown Toronto             2.0                 Coffee Shop   
31       West Toronto             2.0                      Bakery   
32        Scarborough             2.0                  Playground   
33         North York             2.0              Clothing Store   
34         North York             2.0               Metro Station   
36   Downtown Toronto             2.0                 Coffee Shop   
37       West Toronto             2.0                         Bar   
38        Scarborough             2.0            Department Store   
39         North York             2.0         Japanese Restaurant   
41       East Toronto             2.0            Greek Restaurant   
42   Downtown Toronto             2.0                 Coffee Shop   
43       West Toronto             2.0                        Café   
44        Scarborough             2.0                    Bus Line   
45         North York             2.0         Martial Arts School   
46         North York             2.0               Grocery Store   
47       East Toronto             2.0                        Park   
48   Downtown Toronto             2.0                 Coffee Shop   
50         North York             2.0                 Pizza Place   
51        Scarborough             2.0         American Restaurant   
53         North York             2.0            Business Service   
54       East Toronto             2.0                 Coffee Shop   
55         North York             2.0              Sandwich Place   
56               York             2.0                  Restaurant   
58        Scarborough             2.0             College Stadium   
59         North York             2.0            Ramen Restaurant   
60         North York             2.0          Athletics & Sports   
62    Central Toronto             2.0                      Garden   
63               York             2.0           Convenience Store   
65        Scarborough             2.0           Indian Restaurant   
67    Central Toronto             2.0                       Hotel   
69       West Toronto             2.0                         Bar   
70          Etobicoke             2.0                 Pizza Place   
71        Sc

**Cluster 4** green colored on the map

In our list of venue categories, there is only one Print Shop listed and it is the most common venue in this neighborhood.

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
11  Etobicoke             3.0            Print Shop   Dumpling Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
11        Discount Store   Distribution Center               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
11      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue 10th Most Common Venue  
11           Yoga Studio     Dim Sum Restaurant

**Cluster 5**  orange colored on the map

The top venue category in this cluster is *Fast Food Restaurant*. Although this venue category appears in our venue list 24 times, it is in the top 10 **only** in this neighborhood. 

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
6  Scarborough             4.0  Fast Food Restaurant           Yoga Studio   

         3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
6  Eastern European Restaurant   Distribution Center               Dog Run   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
6      Doner Restaurant            Donut Shop             Drugstore   

  9th Most Common Venue 10th Most Common Venue  
6   Dumpling Restaurant      Electronics Store

#### Let's display our cluster centers (centroids) by using the top 10 most common venues. We'll display all the centroids in a data frame.

In [120]:
centroids  = kmeans.cluster_centers_
column_names=toronto_grouped.columns[1:]
# create columns according to number of top venues
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = []
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
centroids_df = pd.DataFrame(columns=columns)

for centroid in centroids:
    # np.argsort sorts in increasing order. Multiplying by -1 ensures we'll get the top 10 indices
    top_10_index=np.argsort(-1*centroid)[:10]
    # column_names are venue category names. By using th top_10_index, we identify the top 10 venue categories for centroids
    dft=pd.DataFrame([column_names[top_10_index]],columns=columns)
    centroids_df=centroids_df.append(dft, ignore_index=True)
    
    

In [122]:
centroids_df = centroids_df.rename_axis('Cluster Labels').reset_index()
centroids_df

Cluster Labels 1st Most Common Venue       2nd Most Common Venue  \
0               0                  Park           Convenience Store   
1               1        Baseball Field  Construction & Landscaping   
2               2           Coffee Shop                 Pizza Place   
3               3            Print Shop           Accessories Store   
4               4  Fast Food Restaurant           Accessories Store   

  3rd Most Common Venue 4th Most Common Venue  \
0                Bakery              Bus Line   
1     Accessories Store   Monument / Landmark   
2                  Café        Sandwich Place   
3        Massage Studio        Medical Center   
4   Martial Arts School        Massage Studio   

             5th Most Common Venue       6th Most Common Venue  \
0                       Playground                     Airport   
1  Molecular Gastronomy Restaurant  Modern European Restaurant   
2                              Bar               Grocery Store   
3         Mediterranean Restaurant                 Men's Store   
4                   Medical Center    Mediterranean Restaurant   

  7th Most Common Venue 8th Most Common Venue       9th Most Common Venue  \
0     Food & Drink Shop                 Trail  Construction & Landscaping   
1     Mobile Phone Shop    Miscellaneous Shop   Middle Eastern Restaurant   
2            Restaurant                  Park                      Bakery   
3         Metro Station    Mexican Restaurant   Middle Eastern Restaurant   
4           Men's Store         Metro Station          Mexican Restaurant   

      10th Most Common Venue  
0               Intersection  
1         Mexican Restaurant  
2                       Bank  
3         Miscellaneous Shop  
4  Middle Eastern Restaurant

### Conclusion

We analyzed Toronto neighborhoods and clustered them based on the venues around them. We can try the following to improve our analysis:

1- Changing (increasing) the number of clusters: Cluster 3 has the majority of the neighborhoods. Increasing the number of clusters in the clustering algorithm may help to better examine the characteristics of the clusters within this big cluster.

2- We can try clustering based on different features:

   - Our clustering algorithm currently uses all the venue categories as input. We displayed the 10-most common   venue categories in each neighborhood. We can try clustering based on the top 10 most common venues only.
   - We can group some venue categories to generate new features and cluster our data based on those new features.
   
3-  Kmeans is fast but it is known to fall in local minima. It is recommended to restart it several times. For the purposes of the analysis in this assignment, we did not work on optimizing the kmeans algorithm. Optimizing the clusters by determining the optimal number of clusters, feature selection, and restarting the kmeans algorithm are some possible future work.